In [3]:
import edgar
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bsoup
import numpy as np
import string
from cleanMDA import extractTable, divide_chunks

In [4]:
res = requests.get('https://etfdb.com/history-of-the-s-and-p-500/#2009')
soup = bsoup(res.content,'html.parser')

In [5]:
tbl = extractTable(soup)
# Remove companies 1980-1999
Lst = tbl[200:]

In [6]:
# Split into chunks of 10 CO per year 
tbl = list(divide_chunks(Lst, 10))
colNames=[i for i in range(2000,2014)]

In [7]:
# Make df
dfTop10 = pd.DataFrame(tbl).transpose()
dfTop10.columns = colNames

In [8]:
# Remove punctuation and uppercase 
for i in colNames:
    dfTop10[i] = dfTop10[i].str.upper().str.strip(string.punctuation).str.strip('\n').str.strip()


In [9]:
dfTop10.replace({'CHEVRON CORP': 'CHEVRON', 'CHEVRON': 'CHEVRON CORP','MICROSOFT': 'MICROSOFT CORP','GENERAL ELECTRIC': 'GENERAL ELECTRIC CO', 'EXXON MOBIL':'EXXON MOBIL CORP', 'PFIZER':'PFIZER INC', 'CITIGROUP':'CITIGROUP INC', 'CISCO SYSTEMS':'CISCO SYSTEMS INC', 'WAL-MART STORES' :'WALMART INC','AIG':'AMERICAN INTERNATIONAL GROUP','MERCK':'MERCK & CO','INTEL':'INTEL CORP','PROCTER & GAMBLE':'PROCTER & GAMBLE CO','AT&T':'AT&T CORP','JPMORGAN CHASE':'JPMORGAN CHASE & CO','GOOGLE':'ALPHABET INC','APPLE':'APPLE INC','BANK OF AMERICA':'BANK OF AMERICA CORP','IBM':'INTL BUSINESS MACHINES CORP','CHEVRON':'CHEVRON CORP','WELLS FARGO':'WELLS FARGO & CO'}, regex=True, inplace=True)

In [10]:
dfCik = pd.read_csv('2000_2013_s&p.csv')
dfCik.drop(['gvkey','gvkeyx','from','thru','tic'], axis=1, inplace=True)
dfCik['co_cik'] = dfCik['co_cik'].fillna(0).astype(int).apply(lambda x: '{0:0>7}'.format(x))
cikDic = pd.Series(dfCik.co_conm.values,index=dfCik.co_cik).to_dict()
cikDic = dict((v,k) for k,v in cikDic.items())

In [11]:
from collections import defaultdict
dic = defaultdict(None)
for yr in colNames:  
    for comp in dfTop10[yr]:
        if comp in cikDic:
            dic[comp] = cikDic[comp]
        else:
            dic[comp] = None
        

In [12]:
for yr in colNames:  
    
    for i in range(len(dfTop10[yr])):
        if type(dfTop10[yr][i])==tuple:
            pass
        else:
            name = dfTop10[yr][i]
            dfTop10[yr][i] =  (name,dic[name])

In [18]:
dfTop10.to_pickle('dfTop10.pkl')

In [13]:
dfTop10

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,"(GENERAL ELECTRIC CO, 0040545)","(GENERAL ELECTRIC CO, 0040545)","(MICROSOFT CORP, 0789019)","(GENERAL ELECTRIC CO, 0040545)","(GENERAL ELECTRIC CO, 0040545)","(GENERAL ELECTRIC CO, 0040545)","(EXXON MOBIL CORP, 0034088)","(EXXON MOBIL CORP, 0034088)","(EXXON MOBIL CORP, 0034088)","(EXXON MOBIL CORP, 0034088)","(EXXON MOBIL CORP, 0034088)","(EXXON MOBIL CORP, 0034088)","(APPLE INC, 0320193)","(APPLE INC, 0320193)"
1,"(EXXON MOBIL CORP, 0034088)","(MICROSOFT CORP, 0789019)","(GENERAL ELECTRIC CO, 0040545)","(MICROSOFT CORP, 0789019)","(EXXON MOBIL CORP, 0034088)","(EXXON MOBIL CORP, 0034088)","(GENERAL ELECTRIC CO, 0040545)","(GENERAL ELECTRIC CO, 0040545)","(WALMART INC, 0104169)","(MICROSOFT CORP, 0789019)","(APPLE INC, 0320193)","(APPLE INC, 0320193)","(EXXON MOBIL CORP, 0034088)","(EXXON MOBIL CORP, 0034088)"
2,"(PFIZER INC, 0078003)","(EXXON MOBIL CORP, 0034088)","(EXXON MOBIL CORP, 0034088)","(EXXON MOBIL CORP, 0034088)","(MICROSOFT CORP, 0789019)","(MICROSOFT CORP, 0789019)","(MICROSOFT CORP, 0789019)","(MICROSOFT CORP, 0789019)","(PROCTER & GAMBLE CO, 0080424)","(WALMART INC, 0104169)","(MICROSOFT CORP, 0789019)","(MICROSOFT CORP, 0789019)","(MICROSOFT CORP, 0789019)","(ALPHABET INC, 1652044)"
3,"(CITIGROUP INC, 0831001)","(CITIGROUP INC, 0831001)","(WALMART INC, 0104169)","(PFIZER INC, 0078003)","(CITIGROUP INC, 0831001)","(CITIGROUP INC, 0831001)","(CITIGROUP INC, 0831001)","(AT&T CORP, 0005907)","(MICROSOFT CORP, 0789019)","(ALPHABET INC, 1652044)","(BERKSHIRE HATHAWAY, 1067983)","(INTL BUSINESS MACHINES CORP, 0051143)","(GENERAL ELECTRIC CO, 0040545)","(MICROSOFT CORP, 0789019)"
4,"(CISCO SYSTEMS INC, 0858877)","(WALMART INC, 0104169)","(PFIZER INC, 0078003)","(CITIGROUP INC, 0831001)","(WALMART INC, 0104169)","(PROCTER & GAMBLE CO, 0080424)","(BANK OF AMERICA CORP, 0070858)","(PROCTER & GAMBLE CO, 0080424)","(AT&T CORP, 0005907)","(APPLE INC, 0320193)","(GENERAL ELECTRIC CO, 0040545)","(CHEVRON CORP, 0093410)","(INTL BUSINESS MACHINES CORP, 0051143)","(GENERAL ELECTRIC CO, 0040545)"
5,"(WALMART INC, 0104169)","(PFIZER INC, 0078003)","(CITIGROUP INC, 0831001)","(WALMART INC, 0104169)","(PFIZER INC, 0078003)","(WALMART INC, 0104169)","(PROCTER & GAMBLE CO, 0080424)","(ALPHABET INC, 1652044)","(JOHNSON & JOHNSON, 0200406)","(JOHNSON & JOHNSON, 0200406)","(WALMART INC, 0104169)","(ALPHABET INC, 1652044)","(CHEVRON CORP, 0093410)","(JOHNSON & JOHNSON, 0200406)"
6,"(MICROSOFT CORP, 0789019)","(INTEL CORP, 0050863)","(JOHNSON & JOHNSON, 0200406)","(INTEL CORP, 0050863)","(BANK OF AMERICA CORP, 0070858)","(BANK OF AMERICA CORP, 0070858)","(WALMART INC, 0104169)","(CHEVRON CORP, 0093410)","(GENERAL ELECTRIC CO, 0040545)","(PROCTER & GAMBLE CO, 0080424)","(ALPHABET INC, 1652044)","(WALMART INC, 0104169)","(JOHNSON & JOHNSON, 0200406)","(CHEVRON CORP, 0093410)"
7,"(AMERICAN INTERNATIONAL GROUP, 0005272)","(INTL BUSINESS MACHINES CORP, 0051143)","(AMERICAN INTERNATIONAL GROUP, 0005272)","(AMERICAN INTERNATIONAL GROUP, 0005272)","(JOHNSON & JOHNSON, 0200406)","(JOHNSON & JOHNSON, 0200406)","(JOHNSON & JOHNSON, 0200406)","(JOHNSON & JOHNSON, 0200406)","(CHEVRON CORP, 0093410)","(INTL BUSINESS MACHINES CORP, 0051143)","(CHEVRON CORP, 0093410)","(GENERAL ELECTRIC CO, 0040545)","(AT&T CORP, 0005907)","(PROCTER & GAMBLE CO, 0080424)"
8,"(MERCK & CO, 0310158)","(AMERICAN INTERNATIONAL GROUP, 0005272)","(INTL BUSINESS MACHINES CORP, 0051143)","(CISCO SYSTEMS INC, 0858877)","(AMERICAN INTERNATIONAL GROUP, 0005272)","(AMERICAN INTERNATIONAL GROUP, 0005272)","(PFIZER INC, 0078003)","(WALMART INC, 0104169)","(PFIZER INC, 0078003)","(AT&T CORP, 0005907)","(INTL BUSINESS MACHINES CORP, 0051143)","(BERKSHIRE HATHAWAY, 1067983)","(PROCTER & GAMBLE CO, 0080424)","(JPMORGAN CHASE & CO, 0019617)"
9,"(INTEL CORP, 0050863)","(JOHNSON & JOHNSON, 0200406)","(MERCK & CO, 0310158)","(INTL BUSINESS MACHINES CORP, 0051143)","(INTL BUSINESS MACHINES CORP, 0051143)","(PFIZER INC, 00

In [15]:
top10_MDA = pd.read_pickle('Top10_MDA.pkl')

In [16]:
top10_MDA

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,"(Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND A...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND AN...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL ST...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL..."
1,"(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Alphabet Inc. Class A, None)","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIA...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSIO...","(WALMART INC, None)","(GENERAL ELECTRIC CO, None)"
2,"(Microsoft Corp., [7 MANAGEMENT’S DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Alphabet Inc. Class A, None)","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(WALMART INC, None)","(PROCTER & GAMBLE CO, None)","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION..."
3,"(Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(GENERAL ELECTRIC CO, [7 “MANAGEMENT’S DISCUSS...","(INTL BUSINESS MACHINES CORP, None)","(BERKSHIRE HATHAWAY, [ 7 MANAGEMENTS DISCUSSI...","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSIO...","(AT&T CORP, None)"
4,"(Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Alphabet Inc. Class A, None)","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(INTL BUSINESS MACHINES CORP, None)","(CHEVRON CORP, [15. EXHIBITS, ...","(GENERAL ELECTRIC CO, [7 MANAGEMENTS DISCUSS...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(AT&T CORP, None)","(PROCTER & GAMBLE CO, None)"
5,"(Berkshire Hathaway Inc, [7 MANAGEMENTS DISC...","(Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN...","(Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN...","(Johnson & Johnson, None)","(General Electric, [7 “MANAGEMENT’S DISCUSSION...","(CHEVRON CORP, [15. EXHIBITS, FINANCIAL STATEM...","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(WALMART INC, None)","(JOHNSON & JOHNSON, None)","(JOHNSON & JOHNSON, None)","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ..."
6,"(Johnson & Johnson, [7MANAGEMENT’S DISCUSSION ...","(Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ...","(Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ...","(Wells Fargo, None)","(Johnson & Johnson, None)","(JOHNSON & JOHNSON, None)","(WALMART INC, None)","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(PROCTER & GAMBLE CO, None)","(GENERAL ELECTRIC CO, [7 MANAGEMENT’S DISCUSSI...","(CHEVRON CORP, [15. EXHIBITS, ..."
7,"(JPMorgan Chase & Co., [601 OF REGULATION S-K....","(Johnson & Johnson, [7MANAGEMENT’S DISCUSSION ...","(General Electric, [ 22 MANAGEMENT'S DIS...","(Wal-Mart Stores Inc, None)","(Wal-Mart Stores Inc, None)","(AT&T CORP, None)","(GENERAL ELECTRIC CO, [7 “MANAGEMENT’S DISCUSS...","(CHEVRON CORP, [15. EXHIBITS, ...","(INTL BUSINESS MACHINES CORP, None)","(CHEVRON CORP, [15. EXHIBITS, ...","(JOHNSON & JOHNSON, None)"
8,"(Exxon Mobil Corp., [16. FORM 10-K SUMMARY...","(JPMorgan Chase & Co., [601 OF REGULATI